In [70]:
import pandas as pd
import numpy as np
import pymongo
import itertools
import csv
import json
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["flick"]

### Initialize

In [78]:
mycol=mydb["posters_data"]
mycol.drop()
csvfile = open('posters.csv', 'r')
reader = csv.DictReader( csvfile )
header=["movieId","imdb_link","poster"]
for each in reader:
    row={}
    for field in header:
        row[field]=each[field]

    mycol.insert_one(row)



mycol = mydb["genres_data"]
mycol.drop()
#genres_df=pd.read_csv('genres.csv')
genre_json=genres_df.to_json(orient='records')
genre_json=eval(genre_json)
genre_data=genre_json
x = mycol.insert_many(genre_data)


mycol = mydb["movies_data"]
mycol.drop()
movies_df=pd.read_csv('movies.csv')
movies_json=movies_df.to_json(orient='records')
movies_json=eval(movies_json)
movies_data=movies_json
movies_data[0]
x = mycol.insert_many(movies_data)

['http://www.imdb.com/title/tt114709' 'http://www.imdb.com/title/tt113497'
 'http://www.imdb.com/title/tt113228' ...
 'http://www.imdb.com/title/tt82875' 'http://www.imdb.com/title/tt815258'
 'http://www.imdb.com/title/tt79142']
[{"movieId":114709,"imdb_link":"http:\/\/www.imdb.com\/title\/tt114709","title":"Toy Story (1995)","


### New review

In [97]:
#inserting new review
uid=1
movieId=114709
rating=4
rating=float(rating)
mycol = mydb["reviews"]
record={ "uid": uid, "movieId": movieId,"rating":rating }
mycol.insert_one(record)
#generating userInput table
userInput=[]
for x in mycol.find({"uid":uid},{ "_id": 0}):
    userInput.append(x)
userInput=pd.DataFrame(userInput)
userInput=userInput.drop(['uid'], axis = 1) 
#fetching genres data from database
mycol = mydb["genres_data"]
data=[]
for x in mycol.find({},{"_id":0}):
    data.append(x)
genres_df=pd.DataFrame(data)
#generating userProfile
userGenre=genres_df[genres_df['movieId'].isin(userInput['movieId'].tolist())]
userGenre.drop('movieId',1,inplace=True)
userGenre.reset_index(drop=True)
userProfile = userGenre.transpose().dot(userInput.rating.values)
#getting new recommendations
genreTable=genres_df.copy()
genreTable.set_index('movieId',inplace=True)
recommend_df=((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommend_df.sort_values(ascending=False,inplace=True)
mycol = mydb["user_recommendation_data"]
recommendation_json=eval(recommend_df.to_json())
recommendation_data={"uid":uid,"recommendation_data":recommendation_json}
mycol.delete_one({"uid":uid})
x = mycol.insert_one(recommendation_data)
recommend_df.head()

movieId
114709    1.0
452598    1.0
371606    1.0
115641    1.0
250223    1.0
dtype: float64

###    Recommendation

In [98]:
mycol = mydb["user_recommendation_data"]
x=mycol.find_one({"uid":1},{"_id":0})
data=x['recommendation_data']
# recommend_df=pd.Series(data)
# recommend_df.head()
data=dict(itertools.islice(data.items(),5))
movies_id=list(data.keys())

mycol=mydb['movies_data']
movies_data=[]
for i in mycol.find({},{"_id":0}):
    if(str(i["movieId"]) in movies_id):
        movies_data.append(i)
        
        
mycol = mydb["posters_data"]
for movie in movies_data:
    data=mycol.find_one({"movieId":str(movie.get('movieId'))},{"_id":0})
    movie['poster']=data['poster']
    movie['imdb_link']=data['imdb_link']
movies_data

[{'movieId': 114709,
  'title': 'Toy Story (1995)',
  'imdb_score': 8.3,
  'genres': 'Animation|Adventure|Comedy',
  'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_UX182_CR0,0,182,268_AL_.jpg',
  'imdb_link': 'http://www.imdb.com/title/tt114709'},
 {'movieId': 115641,
  'title': 'Beavis and Butt-Head Do America (1996)',
  'imdb_score': 6.8,
  'genres': 'Animation|Adventure|Comedy',
  'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BNWY3NmNlYzYtNzY5Yy00Y2Q1LWE1NWEtYzQ2NDM4YTMyYjVmXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg',
  'imdb_link': 'http://www.imdb.com/title/tt115641'},
 {'movieId': 250223,
  'title': 'Asterix and Obelix Meet Cleopatra (2002)',
  'imdb_score': 6.6,
  'genres': 'Adventure|Comedy|Family',
  'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMTg2ODU0MTA1NF5BMl5BanBnXkFtZTcwNzE2MDY5MQ@@._V1_UY268_CR3,0,182,268_AL_.jpg',
  'im